<a href="https://colab.research.google.com/github/choi-yh/Pyhon_ML_Guide/blob/master/09_05_%EC%BD%98%ED%85%90%EC%B8%A0_%EA%B8%B0%EB%B0%98_%ED%95%84%ED%84%B0%EB%A7%81_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TMDB 5000 영화 데이터 세트 사용  
https://www.kaggle.com/tmdb/tmdb-movie-metadata

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링

* 콘텐츠 기반 필터링 : 사용자가 어떤 아이템 속성을 좋아한다면 해당 속성에 대해서 비슷한 평점을 매긴 아이템을 추천하는 방식 
* 장르 컬럼 유사도를 기반으로 영화 추천

### 데이터 로딩 및 가공

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [8]:
movies = pd.read_csv('/gdrive/MyDrive/Python/파이썬_머신러닝_완벽_가이드/data/tmdb_5000/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(2)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


콘텐츠 기반 필터링에 사용할 주요 컬럼만 추출해서 DataFrame 생성하기  
* id
* title : 영화 제목
* genres : 장르
* vote_average : 평균 평점
* vote_count : 평점 투표 수
* popularity : 영화 인기도
* keywords : 영화 설명 주요 키워드
* overview : 영화 개요

In [23]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [24]:
# 값이 dictionary로 되어있는 컬럼(genres, keyword) 확인
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [25]:
# 컬럼을 분해해서 문자열로 파이썬 리스트 객체로 추출 (해당 컬럼의 값이 str 형태임)
# 리스트의 형태로 된 문자열을 리스트로 변환한다고 생각

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [26]:
# 해당 데이터 타입에서 딕셔너리 'name' 값만 추출
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### 장르 콘텐츠 유사도 측정

문자 리스트로 이뤄진 장르별 유사도 측정 방식  
genres를 문자열로 변경 -> CountVetorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교

* 문자열로 변환된 genres 컬럼을 Count 기반으로 피처 벡터화 변환
* genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교  
데이터 세트 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체 생성
* 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [36]:
# 각 genre에 대해 아래처럼 matrix 형태로 변환
genre_mat.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
# 장르 matrix의 코사인 유사도 계산

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 **정렬된 행렬의 위치 인덱스 값** 추출 (비교 대상 행 내에서 정렬된 위치 인덱스)

In [40]:
# numpy argsort 이용해서 정렬된 인덱스 리턴
# 0번 레코드는 3494번째 레코드와의 코사인 유사도가 가장 높음

genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:2])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]]


### 장르 콘텐츠 필터링을 이용한 영화 추천

In [49]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    """
    장르 유사도에 따라 영화를 추천하는 함수
    Args:
        df(pd.DataFrame): movies_df DataFrame
        sorted_ind(np.array): genre_sim_sorted_ind. 레코드별 장르 코사인 유사도 인덱스
        title_name(str): 고객이 선정한 추천 기준이 되는 영화 제목
        top_n(int): 추천할 영화 건수 (default=10)

    Return:
        pd.DataFrame: 추천 영화 정보를 가지는 DataFrame
    """

    # 인자로 입력된 movies_df DataFrame에서 title_name으로 입력된 값을 추출
    title_movie = df[df['title'] == title_name]

    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :top_n]

    # 추출된 top_n index 출력. top_n index는 2차원 데이터
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [50]:
# 대부와 비슷한 영화 추천

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


* 'The Godfather: Part II'와 'GoodFellas'는 비슷한 영화로 잘 추천되었음.  
* 'Light Sleeper', 'Mi America', 'Kids'는 실제로 비슷하지 않은 영화


많은 후보군을 선정한 뒤에 영화 평점에 따라 필터링  

* `vote_average` 컬럼을 사용하는데 이것만을 기준으로 정렬한다면 `vote_count`가 낮은 (한두명이 평점을 매겼을 경우 bias 하게 됨) 영화가 존재  
(`vote_count`에 threshold를 정할 수도 있겠다.) 

In [51]:
# 평점과 평점 카운트를 확인
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


왜곡된 평점 데이터 -> 영화 평점 사이트 IMDB에서 Weighted Rating 방식 사용
$$ Weighted Rating = (\frac{v}{v+m}) \cdot R + (\frac{m}{v+m}) \cdot C $$
* v : 개별 영화에 평점을 투표한 횟수 (`vote_count`)
* m : 평점을 부여하기 위한 최소 투표 횟수 (가중치)
* R : 개별 영화에 대한 평균 평점 (`vote_average`)
* C : 전체 영화에 대한 평균 평점 (`df['vote_average'].mean()`)

In [53]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(f'C: {C:.3f}, m: {m:.3f}')

C: 6.092, m: 370.200


In [56]:
# 가중 평점 계산

percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    '''기존 평점을 가중 평점으로 계산하는 함수'''
    v = record['vote_count']
    R = record['vote_average']

    return ( (v/(v+m)) * R ) + ( (m / (v+m) * C) )

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [58]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [59]:
# weighted_vote 기준으로 다시 추천

def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]

    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


장르만 가지고 유사도를 계산했기 때문에 한계가 명확하다. (해당 장르를 좋아한다고 해서 그 영화를 볼 것이라고 생각하는건 너무 단편적인 생각)
